In [1]:
import os
import sys
import pandas as pd
import nltk, re, string, collections

from nltk import ngrams
from pandas import read_csv
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from nltk.stem import WordNetLemmatizer 


pd.set_option("display.max_columns", 100)

# Se pueden usar 'english' o 'spanish'
stop_words = set(stopwords.words('spanish'))  
lemmatizer = WordNetLemmatizer() 


In [2]:
fileName = 'facts_training_dataset_1617463082875'

# Load dataset inicial
# El objetivo es procesar primero los campos de commits, adrs y chat/emails y dejar solo las palabras mas relevantes
df_ = read_csv('/Users/borisperezg/rebelmodels_storing/datasets_to_train/'+fileName+'.csv', delimiter="|")

In [3]:
df_

,drivertype,goaltype,sourceelementname,sourceelementtype,layersource,targetelementname,targetelementtype,layertarget,isnewelement,iscyclic,facttype,relatontype,actiontype,incoming,outcoming,ratiolinks,mostlinkedlayer,commitlogs,adrlogs,chatlogs,property,propertynewvalue,propertyoldvalue,atdcause,affectedqa
0,cost reduction,process automation,outpatient care collaboration,collaboration,business,NaN,NaN,NaN,NaN,False,element,NaN,create,3,1,0.19,business,the business elements required to accomplish ...,NaN,NaN,NaN,NaN,NaN,noatd,none
1,cost reduction,process automation,pre-service period,process,business,NaN,NaN,NaN,NaN,False,element,NaN,create,0,3,0.14,business,the business elements required to accomplish ...,NaN,NaN,NaN,NaN,NaN,noatd,none
2,cost reduction,process automation,registration process,NaN,NaN,NaN,NaN,NaN,NaN,False,element,NaN,create,1,0,0.05,business,the business elements required to accomplish ...,NaN,NaN,NaN,NaN,NaN,noatd,none
3,cost reduction,process automation,service period,process,business,NaN,NaN,NaN,NaN,False,element,NaN,create,1,5,0.29,business,the business elements required to accomplish ...,NaN,NaN,NaN,NaN,NaN,noatd,none
4,cost reduction,process automation,treatment process,NaN,NaN,NaN,NaN,NaN,NaN,False,element,NaN,create,2,0,0.10,business,the business elements required to accomplish ...,NaN,NaN,NaN,NaN,NaN,noatd,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,cost reduction,process automation,assignment inbound queue,interface,application,assignment topic collaboration,collaboration,application,True,False,relation,serving,create,0,0,0.00,application,the business elements required to accomplish ...,"chosen option: ""implement queue for distribute...",NaN,NaN,NaN,NaN,noatd,none
71,cost reduction,process automation,queue application component,component,application,waiting outbound queue,interface,application,True,False,relation,composition,create,0,0,0.00,application,the business elements required to accomplish ...,"chosen option: ""implement queue for distribute...",NaN,NaN,NaN,NaN,noatd,none
72,cost reduction,process automation,controls estimator component,component,application,caring practices application,NaN,NaN,False,True,relation,consume,create,0,0,0.00,application,the business elements required to accomplish ...,"chosen option: ""bidirectional connection betwe...",NaN,NaN,NaN,NaN,architectural anti-patterns,evolvability
73,cost reduction,process automation,waiting outbound queue,interface,application,assignment topic collaboration,collaboration,application,True,False,relation,serving,create,0,0,0.00,application,the business elements required to accomplish ...,"chosen option: ""implement queue for distribute...",NaN,NaN,NaN,NaN,noatd,none


In [41]:
# ---------------------------------------------------------------------
# DEFINICION DE FUNCIONES DE REMOCION DE STOP WORDS Y LEMATIZACION
# ---------------------------------------------------------------------

In [42]:
def remove_stopwords(newText):
    word_tokens = word_tokenize(newText.lower())  
  
    filtered_sentence = [w for w in word_tokens if not w in stop_words]  
  
    filtered_sentence = []  
  
    for w in word_tokens:  
        # Excluyendo los stopwords
        if w not in stop_words:  
            # Excluyendo puntuaciones
            if w.isalpha():
                filtered_sentence.append(lemmatizer.lemmatize(w))
            
    return " ".join(filtered_sentence)

def bestNGrams(newText):

    my_ngrams = ngrams(newText.split(), 2)
    esBigramFreq = collections.Counter(my_ngrams)
    
    xyz = ''
    
    for letter, count in esBigramFreq.most_common(5):
        xyz += ' '.join(letter)
        xyz += ' '
    
    return xyz

In [43]:
# ------------------------------------------------------------
# PROCESAMIENTO DE CAMPOS COMMITLOGS, ADR Y CHATLOGS
# ------------------------------------------------------------

In [44]:
df_[['commitlogs']] = df_[['commitlogs']].fillna(value='NoData')
df_['commitlogs_lemma'] = df_.commitlogs.apply(remove_stopwords)
df_['commitlogs_ngrams'] = df_.commitlogs_lemma.apply(bestNGrams)

In [45]:
df_[['chatlogs']] = df_[['chatlogs']].fillna(value='NoData')
df_['chatlogs_lemma'] = df_.chatlogs.apply(remove_stopwords)
df_['chatlogs_ngrams'] = df_.chatlogs_lemma.apply(bestNGrams)

In [46]:
df_[['adrlogs']] = df_[['adrlogs']].fillna(value='NoData')
df_['adrlogs_lemma'] = df_.adrlogs.apply(remove_stopwords)
df_['adrlogs_ngrams'] = df_.adrlogs_lemma.apply(bestNGrams)

In [47]:
# ------------------------------------------------------------
# BORRANDO COLUMNAS YA PROCESADAS
# ------------------------------------------------------------

In [48]:
df_.drop(['commitlogs'], axis=1, inplace = True)
df_.drop(['commitlogs_lemma'], axis=1, inplace = True)

df_.drop(['chatlogs'], axis=1, inplace = True)
df_.drop(['chatlogs_lemma'], axis=1, inplace = True)

df_.drop(['adrlogs'], axis=1, inplace = True)
df_.drop(['adrlogs_lemma'], axis=1, inplace = True)

In [49]:
# ------------------------------------------------------------------------------
# SE ALMACENA EL ARCHIVO LISTO PARA SER USADO PARA ENTRENAR O IDENTIFICAR
# ------------------------------------------------------------------------------

In [50]:
df_.to_csv('/Users/borisperezg/rebelmodels_storing/datasets_to_train/'+fileName+'_pa.csv', index=False, index_label=True)

In [ ]:
df_